In [6]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
from energy_cycle import energy_generating_cycle_test

# read model for E. coli BL21
bl21 = read_sbml_model('../GEMs/iHK1487.xml')

# SAL9 module

In [7]:
SAL9 = bl21.copy()

## Metabolites

In [8]:
#L-arogenate
Largn_c = Metabolite(
    'Largn_c',
    formula='C10H13NO5',
    name='L-Arogenate',
    compartment='c')

#4-hydroxyphenyllactate
hpl34_c = Metabolite(
    '34hpl_c',
    formula='C9H10O4',
    name='3,4-Hydroxyphenyl-lactate',
    compartment='c')

#3,4-dihydroxyphenylpyruvate
dhpp34_c = Metabolite(
    '34dhpp_c',
    formula='C9H8O5',
    name='3,4-Dihydroxyphenylpyruvate',
    compartment='c')

#3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_c = Metabolite(
    'saa_c',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='c')

#external 3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_e = Metabolite(
    'saa_e',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='e')

#loading existing metabolites for bl21

#prephenate
pphn_c = bl21.metabolites.get_by_id("pphn_c")

#HPP
hpp34_c = bl21.metabolites.get_by_id("34hpp_c")

#tyrosine 
tyr__L_c = bl21.metabolites.get_by_id("tyr__L_c")

#Oxaloacetate
oaa_c = bl21.metabolites.get_by_id("oaa_c")

#Aspartate
asp = bl21.metabolites.get_by_id("asp__L_c")

#NADH
nadh_c = bl21.metabolites.get_by_id("nadh_c")

#NAD+
nad_c = bl21.metabolites.get_by_id("nad_c")

#H+
h_c = bl21.metabolites.get_by_id("h_c")

#NADPH
nadph_c = bl21.metabolites.get_by_id("nadph_c")

#NADP+
nadp_c = bl21.metabolites.get_by_id("nadp_c")

#H20
h2o_c = bl21.metabolites.get_by_id("h2o_c")

#O2
o2_c = bl21.metabolites.get_by_id("o2_c")

## Reactions

In [9]:
#prep -> l-arogenate
PPNARO = Reaction('PPNARO')
PPNARO.name = 'L-arogenate:2-oxoglutarate aminotransferase'
PPNARO.subsystem = 'Phenylalanine, tyrosine and tryptophan biosynthesis'
PPNARO.lower_bound = -1000  # This is the default
PPNARO.upper_bound = 1000  # This is the default
#Oxaloacetate + L-Arogenate <=> L-Aspartate + Prephenate
PPNARO.add_metabolites({
    oaa_c: -1.0,
    Largn_c: -1.0,
    asp: 1.0,
    pphn_c: 1.0
})
PPNARO.gene_reaction_rule = 'ECD_03926' #tyrB

SAL9.add_reactions([PPNARO])

In [10]:
#HPP -> 4-hydroxyphenyllactate
DLDH = Reaction('DLDH')
DLDH.name = 'D-lactate dehydrogenase'
DLDH.subsystem = 'SAA module'
DLDH.lower_bound = -1000  # This is the default
DLDH.upper_bound = 1000  # This is the default
#3,4-Hydroxyphenyllactate + NAD+ <=> 3,4-Hydroxyphenylpyruvate + NADH + H+
DLDH.add_metabolites({
    hpl34_c: -1.0,
    nad_c: -1.0,
    hpp34_c: 1.0,
    nadh_c: 1.0,
    h_c: 1.0
})
#DLDH.gene_reaction_rule = 'ldh' #synthetic ldh gene

SAL9.add_reactions([DLDH])

In [11]:
#HPP -> 3,4-dihydroxyphenylpyruvate
HPPHD = Reaction('HPPHD')
HPPHD.name = '4-hydroxyphenylacetate 3-hydroxylase'
HPPHD.subsystem = 'SAA module'
HPPHD.lower_bound = -1000  # This is the default
HPPHD.upper_bound = 1000  # This is the default
#4-hydroxyphenylpyruvate + Oxygen + NADH + H+ <=> 3,4-dihydroxyphenylpyruvate + NAD+ + H2O
HPPHD.add_metabolites({
    hpp34_c: -1.0,
    o2_c: -1.0,
    nadh_c: -1.0,
    h_c: -1.0,
    dhpp34_c: 1.0,
    nad_c: 1.0,
    h2o_c: 1.0
})
#HPPHD.gene_reaction_rule = '(hpaB and hpaC)' #synthetic hpaBC gene

SAL9.add_reactions([HPPHD])

In [12]:
#3,4-dihydroxyphenylpyruvate -> salvianic acid A
DHPPSA = Reaction('DHPPSA')
DHPPSA.name = '(R)-3-(4-hydroxyphenyl)lactate:NADP+ oxidoreductase'
DHPPSA.subsystem = 'SAA module'
DHPPSA.lower_bound = -1000  # This is the default
DHPPSA.upper_bound = 1000  # This is the default
#3,4-Dihydroxyphenyllactate + NADP+ <=> 3,4-Dihydroxyphenylpyruvate + NADPH + H+
DHPPSA.add_metabolites({
    saa_c: -1.0,
    nadp_c: -1.0,
    dhpp34_c: 1.0,
    nadph_c: 1.0,
    h_c: 1.0
})
#DHPPSA.gene_reaction_rule = 'ldh' #synthetic ldh gene

SAL9.add_reactions([DHPPSA])

In [13]:
#4-hydroxyphenyllactate -> salvianic acid A
HPLSA = Reaction('HPLSA')
HPLSA.name = '(R)-3-(4-hydroxyphenyl)lactate:NADP+ oxidoreductase'
HPLSA.subsystem = 'SAA module'
HPLSA.lower_bound = -1000  # This is the default
HPLSA.upper_bound = 1000  # This is the default
#4-Hydroxyphenyllactate + Oxygen + NADH + H+ <=> 3,4-Dihydroxyphenylacetic acid + NAD+ + H2O
HPLSA.add_metabolites({
    hpl34_c: -1.0,
    o2_c: -1.0,
    nadh_c: -1.0,
    h_c: -1.0,
    saa_c: 1.0,
    nad_c: 1.0,
    h2o_c: 1.0
})
#HPLSA.gene_reaction_rule = '(hpaB and hpaC)' #synthetic ldh gene

SAL9.add_reactions([HPLSA])

In [14]:
#SAA transport reaction
SAAt = Reaction('SAAt')
SAAt.name = 'caffa transport'
SAAt.subsystem = 'RA module'
SAAt.lower_bound = -1000  # This is the default
SAAt.upper_bound = 1000  # This is the default
SAAt.add_metabolites({
    saa_c: -1.0,
    saa_e: 1.0
})

SAL9.add_reactions([SAAt])

In [17]:
#exhange reactions for SA
SAL9.add_boundary(SAL9.metabolites.get_by_id("saa_e"), type="exchange")

Reaction identifier,EX_saa_e
Name,"3,4-Dihydroxyphenyllactic acid exchange"
Memory address,0x123e8a5c0
Stoichiometry,"saa_e <=> 3,4-Dihydroxyphenyllactic acid <=>"
GPR,
Lower bound,-1000.0
Upper bound,1000.0


# Testing model

In [20]:
#testig for energy-generating cycles
energy_generating_cycle_test(SAL9)

ATPM flux: 0.0


,fluxes,reduced_costs
ALATA_D2,0.0,1.564465e-01
SHCHD2,0.0,-2.000000e+00
CPPPGO,0.0,-2.000000e+00
GTHOr,0.0,1.221245e-15
DHORD5,0.0,-2.000000e+00
...,...,...
HPPHD,0.0,-2.000000e+00
DHPPSA,0.0,2.000000e+00
HPLSA,0.0,-2.000000e+00
SAAt,0.0,2.000000e+00


In [18]:
SAL9.reactions.EX_saa_e.lower_bound = 0

In [19]:
from cobra.flux_analysis import flux_variability_analysis

SAL9.metabolites.saa_c.summary(fva=0.95)

Percent,Flux,Range,Reaction,Definition
100.00%,5.546,[-4.709; 1000],HPLSA,34hpl_c + h_c + nadh_c + o2_c <=> h2o_c + nad_c + saa_c
Percent,Flux,Range,Reaction,Definition
100.00%,-5.546,[-1000; 4.709],DHPPSA,nadp_c + saa_c <=> 34dhpp_c + h_c + nadph_c
0.00%,0,[-0.3767; 0],SAAt,saa_c <=> saa_e


In [34]:
SAL9.metabolites.saa_e.summary(fva=0.95)

Percent,Flux,Range,Reaction,Definition
,0,[0; 0.3767],SAAt,saa_c <=> saa_e
Percent,Flux,Range,Reaction,Definition
,0,[-0.3767; 0],EX_saa_e,saa_e -->
